<a href="https://www.kaggle.com/code/nishithp/oncotrack-server?scriptVersionId=232010211" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Installing Dependencies

In [1]:
!sudo apt install lshw clpeak -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  pci.ids usb.ids
The following NEW packages will be installed:
  clpeak lshw pci.ids usb.ids
0 upgraded, 4 newly installed, 0 to remove and 129 not upgraded.
Need to get 835 kB of archives.
After this operation, 3,128 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 lshw amd64 02.19.git.2021.06.19.996aaad9c7-2build1 [321 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 usb.ids all 2022.04.02-1 [219 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 clpeak amd64 1.1.0-2 [44.8 kB]
Fetched 835 kB in 0s (2,798 kB/s) 
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/per

### Ngrok

In [2]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

--2025-04-03 16:00:55--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 99.83.220.108, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9372184 (8.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.94M  17.0MB/s    in 0.5s    

2025-04-03 16:00:56 (17.0 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [9372184/9372184]



In [3]:
!sudo tar -xvzf ngrok-v3-stable-linux-amd64.tgz -C /usr/local/bin 

ngrok


In [4]:
from kaggle_secrets import UserSecretsClient
import os

user_secrets = UserSecretsClient()
token = user_secrets.get_secret("NGROK_AUTH_TOKEN")
domain = user_secrets.get_secret("NGROK_DOMAIN")

In [5]:
!pip install flask
!pip install pyngrok

In [20]:
!ngrok config add-authtoken $token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
# run_in_background(f"ngrok http --host-header=localhost:5000 --domain {domain} 5000")

In [8]:
from pyngrok import ngrok, conf

conf.get_default().region = "us"

### Unsloth

In [9]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [10]:
import torch
import subprocess
import json
from flask import Flask, request, jsonify
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [11]:
def run_in_background(s):
    subprocess.Popen(s.split())

## Pulling the Fine Tuned Model from Hugging Face

In [12]:
model_name = "tsa_oral_cancer_data_extraction_Meta-Llama-3.1-8B-bnb-4bit_10e"

In [13]:
max_seq_length = 14336 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [14]:
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=f"NishithP2004/{model_name}",  # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit
    )
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

## Alpaca Prompt Template

In [15]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [16]:
instruction = "Extract the essential details from the provided text containing a patient’s oral cancer data. Format the output as a semi-colon separated list of key-value pairs, strictly adhering to the following structure:\nfeature1:value1;feature2:value2;… \nExample: age:40;sex:1;subsite:2;Pathology:3;… \nEnsure precise and efficient extraction by applying the given mappings accurately.\nMappings: 1. age\n2. sex: {male: 1, female: 2}\n3. subsite: {tongue: 1, BM: 2, FOM: 3, alveolus: 4, hard palate: 5, RMT: 6, lips: 7}\n4. pathology: {WDSCC: 1, Verrucous: 2, MDSCC: 3, PDSCC: 4, others: 5, ca in situ: 6}\n5. clinicalT8thed: {T1: 1, T2: 2, T3: 3, T4a: 4, T4b: 5}\n6. nodal8thed: {No nodes: 0, N1: 1, N2a: 2, N2b: 3, N2c: 4, N3a: 5, N3b: 6}\n7. pt8thed: {T1: 1, T2: 2, T3: 3, T4a: 4, T4b: 5}\n8. pN8thed: {N0: 0, N1: 1, N2a: 2, N2b: 3, N2c: 4, N3a: 5, N3b: 6}\n9. PNI: {no: 0, yes: 1}\n10. LVI: {no: 0, yes: 1}\n11. Margins: {negative: 0, close: 1, positive: 2}\n12. Boneinv: {no: 0, yes: 1}\n13. skininv: {no: 0, yes: 1}\n14. ENS: {no: 0, yes: 1}\n15. Surgery: {SND 1-3: 1, SND 1-4: 2, MRND: 3, RND: 4}\n16. lateralitysurgery: {ipsilateral: 1, bilateral: 2}\n17. DOS\n18. adjRx: {no: 0, yes: 1}\n19. date of LR\n20. Nodalrecurrencelevel: {ipsilateral: 1, contralateral: 2}\n21. Nodallevel\n22. date of Nodalrecurrence\n23. last FU\n24. stateFU: {NED: 1, AWD: 2, DWD': 3, Dead due to other causes: 4}\nFullforms \n1. BM – Buccal Mucosa\n2. FOM – Floor of Mouth\n3. RMT – Retromolar Trigone\n4. WDSCC – Well-Differentiated Squamous Cell Carcinoma\n5. MDSCC – Moderately Differentiated Squamous Cell Carcinoma\n6. PDSCC – Poorly Differentiated Squamous Cell Carcinoma\n7. PNI – Perineural Invasion\n8. LVI – Lymphovascular Invasion\n9. ENS – Extracapsular Nodal Spread\n10. SND – Selective Neck Dissection\n11. MRND – Modified Radical Neck Dissection\n12. RND – Radical Neck Dissection\n13. DOS – Date of Surgery\n14. adjRx – Adjuvant Therapy\n15. date of LR – Date of Local Recurrence\n16. last FU – Date of Last Follow-Up\n17. state FU – State at Last Follow-Up\n18. Verrucous - Verrucous Carcinoma"

In [17]:
EOS_TOKEN = tokenizer.eos_token

In [18]:
import pandas as pd

def process_output(output_string):
    """Processes a single output string, splitting by ';' and then ':'.
    Returns a dictionary with key-value pairs from the string.
    """
    result = {}
    if pd.isna(output_string) or output_string == '': # handle missing or empty strings
        return result

    for item in output_string.split(';'):
        try:
            key, value = item.split(':', 1)  # Split at the first ':'
            result[key.strip()] = value.strip()
        except ValueError:  # Handle cases where there's no ':'
            print(f"Skipping malformed item: {item}")
    return result

def process_model_output(output_string):
    """Extracts the response from the model's generated output."""
    try:
        print("Output: ", output_string)
        output_string = output_string.split("### Response:\n")[1].split(EOS_TOKEN)[0]
    except IndexError:
        print("Warning: Unexpected output format. Returning empty dictionary.")
        return {}
    return process_output(output_string)

## Flask Server

In [ ]:
# Define Flask app
app = Flask(__name__)

@app.route("/")
def index():
    return "Hello from Kaggle!"

def generate_response(text):
    input_text = alpaca_prompt.format(instruction, text, "")
    inputs = tokenizer([input_text], return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True)
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    result_dict = process_model_output(generated_text)
    print(result_dict)
    return result_dict

@app.route('/extract', methods=['POST'])
def extract():
    data = request.get_json()
    if not data or 'text' not in data:
        return jsonify({"error": "Invalid input. Expected JSON with 'text' field."}), 400
    response = generate_response(data['text'])
    return jsonify({"response": response})

# Run Flask server in the background
def run_flask():
    public_url = ngrok.connect(5000, domain=domain).public_url
    print(f" * Ngrok Tunnel: {public_url}")
    app.run(host="0.0.0.0", port=5000)


if __name__ == '__main__':
    run_flask()

 * Ngrok Tunnel: https://dominant-usually-oyster.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off
Output:  Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Extract the essential details from the provided text containing a patient’s oral cancer data. Format the output as a semi-colon separated list of key-value pairs, strictly adhering to the following structure:
feature1:value1;feature2:value2;… 
Example: age:40;sex:1;subsite:2;Pathology:3;… 
Ensure precise and efficient extraction by applying the given mappings accurately.
Mappings: 1. age
2. sex: {male: 1, female: 2}
3. subsite: {tongue: 1, BM: 2, FOM: 3, alveolus: 4, hard palate: 5, RMT: 6, lips: 7}
4. pathology: {WDSCC: 1, Verrucous: 2, MDSCC: 3, PDSCC: 4, others: 5, ca in situ: 6}
5. clinicalT8thed: {T1: 1, T2: 2, T3: 3, T4a: 4, T4b: 5}
6. nodal8thed: {No nodes: 0, N1: 1, N2a: 2, N2b: 3, N